## torch loss testing 

In [1]:
import torch

In [2]:
Y=torch.tensor([1,2,0,2,3])
Y_true = torch.nn.functional.one_hot( Y, num_classes=5 )
Y_true

tensor([[0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0]])

In [3]:
Y_pred = torch.tensor( [[0.1,0.6,0.1,0.1,0.1],
                        [0.35,0.15,0.3,0.01,0.19],
                        [0.2,0.2,0.3,0.2,0.1],
                        [0.05,0.1,0.75,0.05,0.05],
                        [0.9,0.025,0.025,0.025,0.025]] )

In [5]:
Y_pred.argmax(1)

tensor([1, 2, 2, 2, 0])

In [5]:
torch.log(Y_pred)

tensor([[-2.3026, -0.5108, -2.3026, -2.3026, -2.3026],
        [-1.0498, -1.8971, -1.2040, -4.6052, -1.6607],
        [-1.6094, -1.6094, -1.2040, -1.6094, -2.3026],
        [-2.9957, -2.3026, -0.2877, -2.9957, -2.9957],
        [-0.1054, -3.6889, -3.6889, -3.6889, -3.6889]])

In [23]:
cost = torch.nn.NLLLoss(reduction="sum")
cost(torch.log(Y_pred), Y) 

tensor(7.3008)

In [9]:
torch.nn.NLLLoss()(torch.log(Y_pred), Y)

tensor(1.4602)

In [16]:
(-Y_pred.log() * Y_true).sum(dim=1).mean()

tensor(1.4602)

In [17]:
-(torch.nn.functional.log_softmax(Y_pred, dim=1) * Y_true).sum(dim=1).mean()

tensor(1.4636)

In [19]:
torch.nn.functional.cross_entropy(input=Y_pred, target=Y_true.to(torch.float32))

tensor(1.4636)

## tf loss testing

In [1]:
import tensorflow as tf

In [2]:
Y=tf.constant([1,2,0,2,3],dtype=tf.int32) #int necessary for one_hot_encoding
Y_true=tf.one_hot(Y,5, axis=1)
Y_true

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.]], dtype=float32)>

In [26]:
Y_pred = tf.constant( [[0.1,0.6,0.1,0.1,0.1],
                        [0.35,0.15,0.3,0.01,0.19],
                        [0.2,0.2,0.3,0.2,0.1],
                        [0.05,0.1,0.75,0.05,0.05],
                        [0.9,0.025,0.025,0.025,0.025]] )

NameError: name 'tf' is not defined

In [5]:
bce = tf.keras.losses.CategoricalCrossentropy( from_logits=False, reduction='sum')
cost = bce(Y_true, Y_pred)

In [6]:
cost

<tf.Tensor: shape=(), dtype=float32, numpy=7.3007975>

## torch accuracy

In [42]:
from torcheval.metrics.functional import multiclass_accuracy
Y=torch.tensor([1,2,0,2,3])
Y_true = torch.nn.functional.one_hot( Y, num_classes=5 )
Y_pred = torch.tensor( [[0.1,0.6,0.1,0.1,0.1],
                        [0.30,0.15,0.35,0.01,0.19],
                        [0.2,0.2,0.3,0.2,0.1],
                        [0.05,0.1,0.45,0.05,0.05],
                        [0.9,0.025,0.025,0.025,0.025]] )
multiclass_accuracy(input=Y_pred, target=Y_true.argmax(1))

tensor(0.6000)

## tf one-hot, loss and accuracy integration test 

In [1]:
import tensorflow as tf
import tensorflow.keras as k
import matplotlib.pyplot as plt
import numpy as np
# Add the directory above the current notebook's directory to the system path
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..'))
from datasets import load_spiral_dataset

In [2]:
X_np,Y_np = load_spiral_dataset(N=1000, noise=0.2, K=5) # x->(D,N), y->(1,N)
X=tf.constant(X_np.T, dtype=tf.float32) # x->(N,D)=(1000,2)
Y=tf.constant(Y_np.reshape(-1,1), dtype=tf.int32) # y->(N,1)=(1000,1)

In [3]:
unique_labels = tf.unique(tf.squeeze(Y)).y
unique_labels

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([0, 1, 2, 3, 4])>

In [4]:
# Y.shape ---> TensorShape([1000, 1])
# Y[:,-1].shape or tf.reshape(Y,[-1]).shape ---> TensorShape([1000])
# takes int (1000) tensor and outputs float (1000,5) tensor 
# first reshape Y to (N,) then cast to int8
Y_ohc = tf.one_hot( tf.cast(Y[:,-1],tf.int8), depth=5 )
# Y_ohc[400:600]

In [5]:
label=tf.constant([1,2,0,3,2],dtype=tf.int32) #int necessary for one_hot_encoding
C = 4
Y_true=tf.one_hot(label,C, axis=1)
Y_pred=tf.constant([[0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.]], dtype=tf.float32)
print(Y_pred)

tf.Tensor(
[[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]], shape=(5, 4), dtype=float32)


In [6]:
# Y_pred=tf.cast(Y_pred,tf.int32)
Y_true=tf.cast(Y_true,tf.int32)

In [7]:
# Y_true can be int, float but Y_pred need to be float
bce = k.losses.CategoricalCrossentropy(from_logits=False, reduction='sum')
cost = bce(Y_true,Y_pred)
cost

<tf.Tensor: shape=(), dtype=float32, numpy=48.354286>

In [8]:
# no need to care about int, float
# in 
acc = k.metrics.CategoricalAccuracy()
acc.update_state(Y_true,Y_pred)
acc.result().numpy()

0.4

In [16]:
tf.argmax(Y_pred, axis=-1)

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([1, 1, 1, 3, 0], dtype=int64)>